### Opinion Review 데이터 세트를 이용한 문서 군집화 수행하기

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob ,os

# 아래는 제 컴퓨터에서 압축 파일을 풀어 놓은 디렉토리이니, 여러분의 디렉토리를 설정해 주십시요  
path = r'/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1.0/topics'                     
# path로 지정한 디렉토리 밑에 있는 모든 .data 파일들의 파일명을 리스트로 취합
all_files = glob.glob(os.path.join(path, "*.data"))    
filename_list = []
opinion_text = []

# 개별 파일들의 파일명은 filename_list 리스트로 취합, 
# 개별 파일들의 파일내용은 DataFrame로딩 후 다시 string으로 변환하여 opinion_text 리스트로 취합 
for file_ in all_files:
    # 개별 파일을 읽어서 DataFrame으로 생성 
    df = pd.read_table(file_,index_col=None, header=0,encoding='latin1')
    
    # 절대경로로 주어진 file 명을 가공. 만일 Linux에서 수행시에는 아래 \\를 / 변경. 맨 마지막 .data 확장자도 제거
    filename_ = file_.split('\\')[-1]
    filename = filename_.split('.')[0]

    #파일명 리스트와 파일내용 리스트에 파일명과 파일 내용을 추가. 
    filename_list.append(filename)
    opinion_text.append(df.to_string())

# 파일명 리스트와 파일내용 리스트를  DataFrame으로 생성
document_df = pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()

# 새 섹션

In [4]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english' , \
                             ngram_range=(1,2), min_df=0.05, max_df=0.85 )

#opinion_text 컬럼값으로 feature vectorization 수행
feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  % sorted(inconsistent)


In [7]:
from sklearn.cluster import KMeans

# 5개 집합으로 군집화 수행. 예제를 위해 동일한 클러스터링 결과 도출용 random_state=0 
km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [8]:
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
1,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
2,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
3,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
4,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0


In [ ]:
# 호텔에 대한 리뷰로 군집화됨
document_df[document_df['cluster_label']==0].sort_values(by='filename')

In [10]:
#포터블 전자기기에 대한 리뷰
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
0,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
3,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
5,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
6,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
7,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
10,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
15,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
17,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,headphone jack i got a clear case for it a...,1
27,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1
34,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,1


In [11]:
# 차량용 네비게이션에 대한 리뷰
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
35,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,2
46,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,2
47,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,2


In [12]:
#킨들 리뷰가 하나 섞여 있지만 대부분 호텔에 대한 리뷰
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
20,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
21,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
22,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
23,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
25,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
31,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
33,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
36,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3
40,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,3


In [13]:
#토요타, 혼다 등 대부분 자동차에 대한 리뷰
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
19,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4
29,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4
37,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4
38,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4
39,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4
41,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4
43,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,4


In [14]:
from sklearn.cluster import KMeans

# 3개의 집합으로 군집화 
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_


# 소속 클러스터를 cluster_label 컬럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
39,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
19,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
47,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
46,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
29,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
43,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
41,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
35,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
38,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0
37,/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8...,...,0


### 군집(Cluster)별 핵심 단어 추출하기


In [15]:
cluster_centers = km_cluster.cluster_centers_
print('cluster_centers shape :',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape : (3, 4611)
[[0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.01005322 0.         0.         ... 0.00706287 0.         0.        ]]


In [16]:
# 군집별 top n 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명들을 반환함. 
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}
    
    # cluster_centers array 의 값이 큰 순으로 정렬된 index 값을 반환
    # 군집 중심점(centroid)별 할당된 word 피처들의 거리값이 큰 순으로 값을 구하기 위함.  
    centroid_feature_ordered_ind = cluster_model.cluster_centers_.argsort()[:,::-1] #argsort를 이용하여 큰 순서로 인덱스 정렬
    
    #개별 군집별로 iteration하면서 핵심단어, 그 단어의 중심 위치 상대값, 대상 파일명 입력
    for cluster_num in range(clusters_num):
        # 개별 군집별 정보를 담을 데이터 초기화. 
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num
        
        # cluster_centers_.argsort()[:,::-1] 로 구한 index 를 이용하여 top n 피처 단어를 구함. 
        top_feature_indexes = centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features = [ feature_names[ind] for ind in top_feature_indexes ]
        
        # top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함 
        top_feature_values = cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()
        
        # cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심위치 상대값, 그리고 해당 파일명 입력
        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_feature_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()
        cluster_details[cluster_num]['filenames'] = filenames
        
    return cluster_details


In [17]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('####### Cluster {0}'.format(cluster_num))
        print('Top features:', cluster_detail['top_features'])
        print('Reviews 파일명 :',cluster_detail['filenames'][:7])
        print('==================================================')


In [18]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df,\
                                  feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

####### Cluster 0
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명 : ['/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1']
####### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명 : ['/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/drive/MyDrive/파이썬 머신러닝 완벽 가이드/8장/data/OpinosisDataset1', '/content/driv

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
